In [2]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': "100%",
        'height': 768,
        'scroll': True,
})

{'width': '100%', 'height': 768, 'scroll': True}

In [3]:
import numpy as np

## ზამბახების მონაცემების მომზადება

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()

In [5]:
import sklearn.utils as ut 
X = iris['data']
y = iris['target']

X, y = ut.shuffle(X, y, random_state=0)


n_test = 50


X_test = X[150 - n_test : ]
y_test = y[150 - n_test : ]

X = X[:150 - n_test]
y = y[:150 - n_test]


print(f'{y=}')
print(f'{X.shape=}, {y.shape=}')

print(f'{y_test=}')
print(f'{X_test.shape=}, {y_test.shape=}')

print(f'{iris.feature_names}')

y=array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1,
       1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0])
X.shape=(100, 4), y.shape=(100,)
y_test=array([0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2,
       0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0,
       0, 0, 2, 1, 2, 0])
X_test.shape=(50, 4), y_test.shape=(50,)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [6]:
X = np.c_[X, np.ones(len(X))]
X_test = np.c_[X_test, np.ones(len(X_test))]


In [7]:
import scipy.sparse.linalg as la
import scipy.optimize as opt

WW = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = np.copy(y)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(f'{y=}')
print(f'{label_encoder=}')
print(f'{onehot_encoder=}')
print(f'{integer_encoded=}')
print(f'{onehot_encoded=}')


values_test = np.copy(y_test)

# integer encode
label_encoder_test = LabelEncoder()
integer_encoded_test = label_encoder_test.fit_transform(values_test)
# binary encode
onehot_encoder_test = OneHotEncoder(sparse=False)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder_test.fit_transform(integer_encoded_test)

y=array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1,
       1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0])
label_encoder=LabelEncoder()
onehot_encoder=OneHotEncoder(sparse=False)
integer_encoded=array([[2],
       [1],
       [0],
       [2],
       [0],
       [2],
       [0],
       [1],
       [1],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [2],
       [1],
       [0],
       [0],
       [2],
       [0],
       [0],
       [1],
       [1],
       [0],
       [2],
       [1],
       [0],
       [2],
       [2],
       [1],
       [0],
       [1],
       [1],
       [1],
       [2],
       [0],
       [2],
       [0],
       [0],
       [1],
       [2],
       [2]

In [9]:
import math

def sigmoid(x):
    x[x < -50] = -50
    x[x > 50] = 50
    return 1 / (1 + np.exp(-x)) 

def f(w):
    return ff(w, X, y)
    
def ff(w, x, y):
    return -np.sum(y * np.log(sigmoid(np.matmul(x, w))) + (1 - y) * np.log(sigmoid(-np.matmul(x, w))))

def f_prime(X,y,w):
    p = (sigmoid(np.matmul(X, w)) - y)
    return np.matmul(np.transpose(X), p)

def r_gradient(X,y,ww,f:callable,fprime:callable,epoch,alpha):
    ww_copy =  np.copy(ww)
    for i in range(epoch):
        gradient = fprime(X,y,ww_copy)
        for j in range(len(ww)):
            ww_copy[j] = ww_copy[j] - (alpha * gradient[j])
    return ww_copy

def validate(ww, X:np.ndarray, y:np.ndarray) -> float:
        y_init = sigmoid(np.matmul(X, ww))
        y_pred = np.rint(y_init)
        score = float(sum(y_pred == y)) / float(len(y))
        
        return score

x = np.zeros((3, 5))

print(WW)
for i in range(3):
    print()
    print(f"{i=} ********************************** ")
    print()
    y = onehot_encoded[:,i]
    ww_new =r_gradient(X,y,WW,f,f_prime,300_000,.0001)
    train_accuracy = validate(ww_new,X,y)
    test_accuracy = validate(ww_new,X_test,y_test)
    print(f'{train_accuracy=}')
    print(f'{test_accuracy=}')

[0.1 0.1 0.1 0.1 0.1]

i=0 ********************************** 

train_accuracy=1.0
test_accuracy=0.0

i=1 ********************************** 

train_accuracy=0.74
test_accuracy=0.58

i=2 ********************************** 

train_accuracy=0.98
test_accuracy=0.42
